In [1]:
import cuml
from cupy import asnumpy
from joblib import dump, load
import numpy as np
import cupy as cp
import pandas as pd
import cudf
import time

In [3]:
df = pd.read_csv('diabetes.csv')
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import brier_score_loss as sk_brier_score_loss
#df = df.drop('Species', axis=1)
#y=df.DiabetesPedigreeFunction
y=df.Outcome
X_train, X_test, y_train, y_test = train_test_split(df, y,test_size=0.2)
lr = LogisticRegression()
lr.fit(X_train, y_train)
predictions = lr.predict_proba(X_test)[:,1]
y_test = cp.asarray(y_test) 
predictions = cp.asarray(predictions)

classes = cp.unique(y_test)
def check_length(y_true, y_pred):
    if  len(y_true) == len(y_pred):
        return True
    else:
        return False


def brier_score_loss(y_true, y_prob, *, sample_weight=None, pos_label=0):
    if check_length(y_true, y_prob)==True:
        #labels = cp.unique(y_true)
        if pos_label is None:
            if (cp.array_equal(labels, [0]) or
                    cp.array_equal(labels, [-1])):
                pos_label = 1
            else:
                pos_label = y_true.maximum()
        y_true = cp.asarray(y_true == pos_label, int)
    return float(cp.average((y_true - y_prob) ** 2, weights=sample_weight))


s = time.time()
cu_score = brier_score_loss(y_test, 1-predictions)
e = time.time()
print("Time consumed by cuml: ", e-s)
p = time.time()
sk_score = sk_brier_score_loss(asnumpy(y_test), asnumpy(predictions))

d = time.time()
print("Time consumed by sklearn: ", d-p)

print( " cuml Brier Score Loss: ", cu_score )
print( " sklearn Brier Score Loss : ", sk_score )

Time consumed by cuml:  0.0006012916564941406
Time consumed by sklearn:  0.0007317066192626953
 cuml Brier Score Loss:  0.0008247571243787482
 sklearn Brier Score Loss :  0.0008247571243787482


/opt/conda-environments/rapids-stable/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
